In [1]:
import tkinter
import re
import pickle
import pandas as pd
import tkinter as tk
from tkinter import *
from tkinter import simpledialog
from tkinter import messagebox
import tkinter
from nltk.stem import WordNetLemmatizer, PorterStemmer

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


In [7]:
def preprocess(textdata):
    processedText = []
    # Create Lemmatizer and Stemmer
    wordLemm = WordNetLemmatizer()
    # Regex patterns
    emoji = r"\\u[1F600-1F6FF]"
    urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern = "@[^\s]+"
    alphaPattern = "[^a-zA-Z0-9]"
    sequencePattern = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    for tweet in textdata:
        tweet = tweet.lower()
        # Replace all URls with 'URL'
        tweet = re.sub(urlPattern, " ", tweet)
        # Replace all emojis.
        tweet = re.sub(emoji, " ", tweet)
        # Replace @USERNAME to 'USER'.
        tweet = re.sub(userPattern, " ", tweet)
        # Replace all non alphabets.
        tweet = re.sub(alphaPattern, " ", tweet)
        # Replace 3 or more consecutive letters by 2 letter.
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)
        tweetwords = ""
        for word in tweet.split():
            # Checking if the word is a stopword.
            # If word not in stopwordlist
            if len(word) > 1:

                word = wordLemm.lemmatize(word)

                tweetwords += word + " "
        processedText.append(tweetwords)

    return processedText

def load_models():
    '''
    Replace '..path/' by the path of the saved models.
    '''
    
    file = open('D:/6th semester/Artifical Intelligence Lab/Emotion Prediction/vectoriser-ngram-(1,2).pickle', 'rb')
    vectorizer = pickle.load(file)
    file.close()
    # Load the LR Model.
    file = open('D:/6th semester/Artifical Intelligence Lab/Emotion Prediction/Sentiment-LR.pickle', 'rb')
    LRmodel = pickle.load(file)
    file.close()

    # Load the RF Model.
    file = open('D:/6th semester/Artifical Intelligence Lab/Emotion Prediction/Sentiment-RF.pickle', 'rb')
    RFmodel = pickle.load(file)
    file.close()

     # Load the NB Model.
    file = open('D:/6th semester/Artifical Intelligence Lab/Emotion Prediction/Sentiment-BNB.pickle', 'rb')
    BNB = pickle.load(file)
    file.close()

     # Load the SVC Model.
    file = open('D:/6th semester/Artifical Intelligence Lab/Emotion Prediction/Sentiment-SVC.pickle', 'rb')
    SVCmodel = pickle.load(file)
    file.close()
    
    return vectorizer, LRmodel,RFmodel,BNB,SVCmodel

def predict(vectorizer, model, text):
    # Predict the sentiment
    textdata = vectorizer.transform(preprocess(text))
    sentiment = model.predict(textdata)
    
    # Make a list of text with sentiment.
    data = []
    for text, pred in zip(text,sentiment):
        data.append((text,pred))
        
    # Convert the list into a Pandas DataFrame.
    df = pd.DataFrame(data, columns = ['Text','Sentiment'])
    df = df.replace([0,1,2], ["Negative","Positive","Neutral"])
    return df



In [10]:
root = tk.Tk()

root.geometry("700x500")

name_var = tk.StringVar()


def submit():

    name = name_var.get()
    text = []
    text.append(name)
    vectorizer, LRmodel, RFmodel,BNB,SVCmodel = load_models()
    df = predict(vectorizer, SVCmodel, text)
    data = df['Sentiment']
    tkinter.messagebox.showinfo('Prediction', data.item())

    name_var.set("")


root.configure(bg='#eb345b')
root.title("Emotion Prediction")
root.iconbitmap(r'D:/6th semester/Artifical Intelligence Lab/Emotion Prediction/emotions.ico')

canvas = Canvas(root,width=400, height=330)
canvas.place(x=150, y=60)

title = Label(root, text="Emotion Prediction", font="Bold 26")
title.place(x=180, y=150)

text = Label(root, text="Enter Your Text", font="Bold 16")
text.place(x=200, y=220)

name_entry = tk.Entry(root, textvariable=name_var,
                      font=('calibre', 14, 'normal'))
name_entry.place(x=250, y=250)

submitButtonImage = PhotoImage(file=r"D:/6th semester/Artifical Intelligence Lab/Emotion Prediction/img0.png")
submitbutton = Button(root, image=submitButtonImage, command=submit, border=0)
submitbutton.place(x=250, y=300)


root.mainloop()

